In [3]:
import tensorflow as tf
# tf.enable_eager_execution()
from tensorflow import keras
from tensorflow.keras import  layers


In [4]:
import sys
sys.path.insert(0, '../lib/')
from help_functions import *

#function to obtain data for training/testing (validation)
from extract_patches import get_data_training
sys.path.insert(0, '../lib/networks/')
from fully_conv import get_fully_conv_unet as net_
from fully_conv import attend
from preprocessing import preprocessing

from pixel_dcn import pixel_dcl

In [5]:
print(tf.keras.__version__)

2.1.6-tf


In [6]:
train_img, label_img = preprocessing()


train images/masks shape:
(20, 1, 565, 565)
train images range (min-max): 0.0 - 1.0
train masks are within 0-1

patches per full image: 9500

train PATCHES images/masks shape:
(190000, 1, 48, 48)
train PATCHES images range (min-max): 0.00784313725490196 - 1.0
(190000, 1, 48, 48)
48 48 1
......DONE......
old shape:  (190000, 2304)
new shape:  (190000, 2304, 2)


In [12]:
sess=tf.Session()
def attend_(n_ch,patch_height, patch_width):
    chan = 'channels_first'
    inputs = layers.Input(shape=(n_ch,patch_height, patch_width))
    
    conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same',data_format=chan)(inputs)
    conv1 = layers.Dropout(0.2)(conv1)
    conv1_strided = layers.Conv2D(32, (3, 3), activation='relu', padding='same',strides=2,data_format=chan)(conv1)
    
    conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same',data_format=chan)(conv1_strided)
    conv2 = layers.Dropout(0.2)(conv2)
    conv2_strided = layers.Conv2D(64, (3, 3), activation='relu', padding='same',strides=2,data_format=chan)(conv2)
    
#     conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same',data_format=chan)(conv2_strided)
#     conv3 = layers.Dropout(0.2)(conv3)
#     conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same',data_format=chan)(conv3)
    print(conv2_strided)
    roll = tf.transpose(conv2_strided, [0,2,3,1])
    
    up2 = keras.layers.Lambda(lambda x: pixel_dcl(inputs=x, out_num=128, kernel_size=(2, 2), 
                                     scope='conv_pdcl', d_format='NHWC'))(roll)
    
#     att = keras.layers.Lambda(lambda x: multihead_attention_2d(inputs=roll, total_key_filters=64,
#                                                                total_value_filters=64, output_filters=128, 
#                                                                num_heads=8, training=True,
#                                                                layer_type='SAME', scope='att'))(roll) 
#     roll = tf.transpose(up2, [0,3,1,2])
#     #up2 = UpSampling2D(size=(2, 2), data_format='channels_last')(conv4)
# #     up2 = keras.layers.Concatenate(axis=3)([conv1, roll])
    
    
# #     up1 = layers.UpSampling2D(size=(2, 2), data_format=chan)(roll)
#     up1 = layers.Concatenate(axis=1)([conv2, roll])
#     print('shape up1', up1.shape)
    
#     conv4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same',data_format=chan)(up1)
#     conv4 = layers.Dropout(0.2)(conv4)
#     conv4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same',data_format=chan)(conv4)
    
#     up2 = layers.UpSampling2D(size=(2, 2), data_format=chan)(conv4)
#     print('shape up2', up2.shape)
#     up2 = layers.Concatenate(axis=1)([conv1, up2])
    
#     conv5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same',data_format=chan)(up2)
#     conv5 = layers.Dropout(0.2)(conv5)
#     conv5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same',data_format=chan)(conv5)
    
#     conv6 = layers.Conv2D(2, (1, 1), activation='relu',padding='same',data_format=chan)(conv5)
#     conv6 = layers.Reshape((2,patch_height*patch_width))(conv6)
#     conv6 = layers.Permute((2,1))(conv6)
    
#     conv7 = keras.layers.Lambda(layers.Activation('softmax'))(conv6)
# #     print(conv7)
    model = keras.Model(inputs=inputs, outputs=up2)
    print(model.summary())

    adam = tf.keras.optimizers.Adam(lr=0.001, decay=.01)#, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    return model
model = attend_(1,48,48)

#save model
name_experiment = 'test'
path = '../'+name_experiment+'/'+name_experiment
json_string = model.to_json()
open(path+'_architecture.json', 'w').write(json_string)

checkpoints = keras.callbacks.ModelCheckpoint(filepath=path+'_best_w.h5', 
                                            monitor='val_loss',verbose=2,
                                            save_best_only=True, )

history = model.fit(train_img, label_img, epochs=20,
                    validation_split = 0.1, 
                    shuffle=True, verbose=2, 
                    callbacks=[checkpoints])




Tensor("conv2d_48/Relu:0", shape=(?, 64, 12, 12), dtype=float32)


ValueError: Output tensors to a Model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: Tensor("lambda_8/Relu:0", shape=(?, 24, 24, 128), dtype=float32)

In [7]:
hist = model.fit(patches_imgs_train, patches_masks_train, 
          nb_epoch=50, batch_size=64, 
          verbose=2, shuffle=True, validation_split=0.1) 

Train on 171000 samples, validate on 19000 samples
Epoch 1/50
 - 80s - loss: 0.1724 - acc: 0.9354 - val_loss: 0.1342 - val_acc: 0.9480
Epoch 2/50
 - 77s - loss: 0.1415 - acc: 0.9457 - val_loss: 0.1305 - val_acc: 0.9490
Epoch 3/50
 - 77s - loss: 0.1372 - acc: 0.9471 - val_loss: 0.1296 - val_acc: 0.9491
Epoch 4/50
 - 76s - loss: 0.1349 - acc: 0.9479 - val_loss: 0.1298 - val_acc: 0.9489
Epoch 5/50
 - 76s - loss: 0.1333 - acc: 0.9484 - val_loss: 0.1283 - val_acc: 0.9494
Epoch 6/50


KeyboardInterrupt: 